In [1]:
import pandas as pd
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import copy

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
data = pd.read_csv('Dataset/TC2/train.csv')

In [4]:
data

,velocity_data_x,velocity_data_y,velocity_data_z
0,-0.134012,0.466469,-0.051024
1,-0.202646,0.543844,-0.040844
2,-0.202646,0.543844,-0.040844
3,-0.267686,0.621683,-0.031170
4,-0.299013,0.660611,-0.026600
...,...,...,...
2943,-1.211667,0.883520,-0.000159
2944,-1.211675,0.883522,-0.000159
2945,-1.211683,0.883522,-0.000159
2946,-1.211691,0.883519,-0.000159


In [5]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

df_max_scaled = data.copy()
columns_to_scale = ['velocity_data_x', 'velocity_data_y', 'velocity_data_z']
scaler = MinMaxScaler()

for column in columns_to_scale:
    df_max_scaled[column] = scaler.fit_transform(np.array(df_max_scaled[column]).reshape(-1, 1) )
    
df_max_scaled

,velocity_data_x,velocity_data_y,velocity_data_z
0,0.987465,0.140303,0.000000
1,0.934934,0.193936,0.117364
2,0.934934,0.193936,0.117364
3,0.885153,0.247892,0.228894
4,0.861175,0.274875,0.281580
...,...,...,...
2943,0.162641,0.429388,0.586414
2944,0.162635,0.429389,0.586414
2945,0.162629,0.429389,0.586414
2946,0.162623,0.429387,0.586414


In [6]:
sequence_size = 4
sequences = []

for i in range(len(data) - sequence_size + 1):
    sequence = df_max_scaled.iloc[i:i + sequence_size]
    sequences.append(sequence)

sequences_df = pd.concat(sequences, ignore_index=True)

sequences_df

,velocity_data_x,velocity_data_y,velocity_data_z
0,0.987465,0.140303,0.000000
1,0.934934,0.193936,0.117364
2,0.934934,0.193936,0.117364
3,0.885153,0.247892,0.228894
4,0.934934,0.193936,0.117364
...,...,...,...
11775,0.162623,0.429387,0.586414
11776,0.162635,0.429389,0.586414
11777,0.162629,0.429389,0.586414
11778,0.162623,0.429387,0.586414


In [7]:
x_train = sequences_df.values.tolist()
x_train

[[0.9874652418204904, 0.14030282991533682, 0.0],
 [0.9349336064237478, 0.19393647004348913, 0.11736358500789723],
 [0.9349336064237478, 0.19393647004348913, 0.11736358500789723],
 [0.8851527752583758, 0.24789173872116604, 0.2288935772835748],
 [0.9349336064237478, 0.19393647004348913, 0.11736358500789723],
 [0.9349336064237478, 0.19393647004348913, 0.11736358500789723],
 [0.8851527752583758, 0.24789173872116604, 0.2288935772835748],
 [0.8611754674798148, 0.27487526496231257, 0.28158037330381946],
 [0.9349336064237478, 0.19393647004348913, 0.11736358500789723],
 [0.8851527752583758, 0.24789173872116604, 0.2288935772835748],
 [0.8611754674798148, 0.27487526496231257, 0.28158037330381946],
 [0.8378311355218835, 0.3016993632586517, 0.3325724299334785],
 [0.8851527752583758, 0.24789173872116604, 0.2288935772835748],
 [0.8611754674798148, 0.27487526496231257, 0.28158037330381946],
 [0.8378311355218835, 0.3016993632586517, 0.3325724299334785],
 [0.8152055028330834, 0.32813806182754335, 0.3818

In [41]:
import torch.nn as nn

class LSTMModel_forward(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers,dropout):
        super(LSTMModel_forward, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.lstm_forward = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout).to(self.device)
        self.fc1_forward = nn.Linear(hidden_size, hidden_size).to(self.device)
        self.relu_forward = nn.ReLU()
        self.fc2_forward = nn.Linear(hidden_size, input_size).to(self.device)
        self._init_weights()

    def forward(self, x):
        out_forward, _ = self.lstm_forward(x)
        out_forward = self.fc1_forward(out_forward)
        out_forward = self.relu_forward(out_forward)
        out_forward = self.fc2_forward(out_forward)
        return out_forward
    
    def _init_weights(self):
        def init_weights(m):
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight.data)
                if m.bias is not None:
                    nn.init.constant_(m.bias.data, 0)
            elif isinstance(m, nn.LSTM):
                for name, param in m.named_parameters():
                    if 'weight' in name:
                        nn.init.kaiming_normal_(param.data)
                    elif 'bias' in name:
                        nn.init.constant_(param.data, 0)

In [42]:
input_size = 3
dropout = 0.1
num_layers=1
hidden_size=64
num_epochs=50
learning_rate=0.01

In [43]:
model_1 = LSTMModel_forward(input_size, hidden_size, num_layers,dropout).cuda()
model_1.to(device)

criterion = nn.MSELoss()

optimizer_1 = torch.optim.Adam(model_1.parameters(), lr=0.001)

In [44]:
# Define weight initialization function


In [45]:
last_index = len(x_train) - 4
num_sequences = last_index // 4
predicted_vectors = []

for epoch in range(num_epochs):
    predicted_vectors.clear()
    total_loss = 0

    for idx in range(0, last_index, 4):
        sequence = torch.tensor(x_train[idx: idx + 4]).to(device)
        optimizer_1.zero_grad()

        # Call model with only the sequence
        out_forward = model_1(sequence)

        # Calculate predicted next element and target (moved outside model call)
        predicted_next_element_1 = out_forward[-1, :].view(-1, 3)
        target_next_element = torch.tensor(x_train[idx + 7]).view(-1, 3).to(device)

        # Calculate loss without extra arguments
        loss_forward_1 = criterion(predicted_next_element_1, target_next_element)
        total_loss += loss_forward_1.item()

        loss_forward_1.backward()
        optimizer_1.step()

        if idx % 100 == 0:
            print(f"Epoch {epoch + 1}/{num_epochs}, Sequence {idx // 4 + 1}/{num_sequences}, Loss: {loss_forward_1.item()}")

        predicted_vectors.append(predicted_next_element_1.detach().cpu().numpy())

    # Calculate average loss for the epoch
    avg_loss = total_loss / num_sequences


Epoch 1/50, Sequence 1/2944, Loss: 0.2970002591609955
Epoch 1/50, Sequence 26/2944, Loss: 0.033808447420597076
Epoch 1/50, Sequence 51/2944, Loss: 0.013189723715186119
Epoch 1/50, Sequence 76/2944, Loss: 0.03980724513530731
Epoch 1/50, Sequence 101/2944, Loss: 0.007910975255072117
Epoch 1/50, Sequence 126/2944, Loss: 0.00015548249939456582
Epoch 1/50, Sequence 151/2944, Loss: 0.0001883444783743471
Epoch 1/50, Sequence 176/2944, Loss: 3.314691275591031e-05
Epoch 1/50, Sequence 201/2944, Loss: 8.556802640669048e-06
Epoch 1/50, Sequence 226/2944, Loss: 4.306565642764326e-06
Epoch 1/50, Sequence 251/2944, Loss: 1.0161629688809626e-06
Epoch 1/50, Sequence 276/2944, Loss: 1.9530830286385026e-07
Epoch 1/50, Sequence 301/2944, Loss: 4.039227263774592e-08
Epoch 1/50, Sequence 326/2944, Loss: 1.9530350670038388e-08
Epoch 1/50, Sequence 351/2944, Loss: 3.324916519886756e-08
Epoch 1/50, Sequence 376/2944, Loss: 5.1637229603329615e-08
Epoch 1/50, Sequence 401/2944, Loss: 2.4034182644072644e-08
Epoc

Epoch 2/50, Sequence 501/2944, Loss: 3.687719241085574e-09
Epoch 2/50, Sequence 526/2944, Loss: 4.829825428487311e-09
Epoch 2/50, Sequence 551/2944, Loss: 2.913639884916108e-10
Epoch 2/50, Sequence 576/2944, Loss: 9.583398519197317e-10
Epoch 2/50, Sequence 601/2944, Loss: 1.0490797119899753e-09
Epoch 2/50, Sequence 626/2944, Loss: 2.1085084789262964e-09
Epoch 2/50, Sequence 651/2944, Loss: 2.3717028341252444e-09
Epoch 2/50, Sequence 676/2944, Loss: 5.910365530326089e-10
Epoch 2/50, Sequence 701/2944, Loss: 1.9183352129026332e-10
Epoch 2/50, Sequence 726/2944, Loss: 1.9899341896501e-10
Epoch 2/50, Sequence 751/2944, Loss: 4.214951443426429e-10
Epoch 2/50, Sequence 776/2944, Loss: 9.564036229647854e-10
Epoch 2/50, Sequence 801/2944, Loss: 8.973657372735033e-10
Epoch 2/50, Sequence 826/2944, Loss: 1.4054595265378111e-09
Epoch 2/50, Sequence 851/2944, Loss: 2.063629267468059e-10
Epoch 2/50, Sequence 876/2944, Loss: 1.1166653018923611e-10
Epoch 2/50, Sequence 901/2944, Loss: 8.5722248743813

Epoch 3/50, Sequence 1001/2944, Loss: 5.206620551367891e-11
Epoch 3/50, Sequence 1026/2944, Loss: 1.0717234738555348e-10
Epoch 3/50, Sequence 1051/2944, Loss: 7.341446200959112e-11
Epoch 3/50, Sequence 1076/2944, Loss: 3.7385444739967966e-10
Epoch 3/50, Sequence 1101/2944, Loss: 1.8532968215634327e-10
Epoch 3/50, Sequence 1126/2944, Loss: 3.6185693330637037e-10
Epoch 3/50, Sequence 1151/2944, Loss: 3.71567665524708e-10
Epoch 3/50, Sequence 1176/2944, Loss: 3.859987884879956e-10
Epoch 3/50, Sequence 1201/2944, Loss: 6.290079568316287e-10
Epoch 3/50, Sequence 1226/2944, Loss: 6.430293519876784e-10
Epoch 3/50, Sequence 1251/2944, Loss: 1.3453831382292947e-09
Epoch 3/50, Sequence 1276/2944, Loss: 1.1970348046119739e-09
Epoch 3/50, Sequence 1301/2944, Loss: 1.4609765619510995e-09
Epoch 3/50, Sequence 1326/2944, Loss: 6.806656349667151e-10
Epoch 3/50, Sequence 1351/2944, Loss: 1.186577947009937e-09
Epoch 3/50, Sequence 1376/2944, Loss: 3.8769817911621374e-10
Epoch 3/50, Sequence 1401/2944, L

Epoch 4/50, Sequence 1501/2944, Loss: 1.552968198481608e-09
Epoch 4/50, Sequence 1526/2944, Loss: 1.6589989798587723e-11
Epoch 4/50, Sequence 1551/2944, Loss: 1.0596146182706434e-09
Epoch 4/50, Sequence 1576/2944, Loss: 1.5821418610992288e-12
Epoch 4/50, Sequence 1601/2944, Loss: 3.439382112446765e-11
Epoch 4/50, Sequence 1626/2944, Loss: 1.8143590796437792e-10
Epoch 4/50, Sequence 1651/2944, Loss: 2.12878603633726e-11
Epoch 4/50, Sequence 1676/2944, Loss: 1.1131315869050695e-09
Epoch 4/50, Sequence 1701/2944, Loss: 2.9499011566791467e-10
Epoch 4/50, Sequence 1726/2944, Loss: 7.692809930737177e-12
Epoch 4/50, Sequence 1751/2944, Loss: 2.1942152261811287e-10
Epoch 4/50, Sequence 1776/2944, Loss: 6.581639400149442e-11
Epoch 4/50, Sequence 1801/2944, Loss: 3.8969716342762695e-11
Epoch 4/50, Sequence 1826/2944, Loss: 5.782740397641817e-10
Epoch 4/50, Sequence 1851/2944, Loss: 7.16333659056545e-11
Epoch 4/50, Sequence 1876/2944, Loss: 6.549380482390177e-10
Epoch 4/50, Sequence 1901/2944, Lo

Epoch 5/50, Sequence 2001/2944, Loss: 1.8499335396882088e-10
Epoch 5/50, Sequence 2026/2944, Loss: 0.00011077639646828175
Epoch 5/50, Sequence 2051/2944, Loss: 0.0020030552987009287
Epoch 5/50, Sequence 2076/2944, Loss: 0.024224480614066124
Epoch 5/50, Sequence 2101/2944, Loss: 0.040418632328510284
Epoch 5/50, Sequence 2126/2944, Loss: 0.009082055650651455
Epoch 5/50, Sequence 2151/2944, Loss: 0.00034318328835070133
Epoch 5/50, Sequence 2176/2944, Loss: 3.697740248753689e-05
Epoch 5/50, Sequence 2201/2944, Loss: 0.00016838894225656986
Epoch 5/50, Sequence 2226/2944, Loss: 0.00013563247921410948
Epoch 5/50, Sequence 2251/2944, Loss: 8.175696711987257e-05
Epoch 5/50, Sequence 2276/2944, Loss: 5.0430637202225626e-05
Epoch 5/50, Sequence 2301/2944, Loss: 1.9497516404953785e-05
Epoch 5/50, Sequence 2326/2944, Loss: 6.786889116483508e-06
Epoch 5/50, Sequence 2351/2944, Loss: 2.0138020317972405e-06
Epoch 5/50, Sequence 2376/2944, Loss: 2.8790122996724676e-07
Epoch 5/50, Sequence 2401/2944, Lo

...

Epoch 46/50, Sequence 301/2944, Loss: 0.01010734960436821
Epoch 46/50, Sequence 326/2944, Loss: 0.007433589082211256
Epoch 46/50, Sequence 351/2944, Loss: 0.005366429220885038
Epoch 46/50, Sequence 376/2944, Loss: 0.0038129412569105625
Epoch 46/50, Sequence 401/2944, Loss: 0.0026938009541481733
Epoch 46/50, Sequence 426/2944, Loss: 0.001897535752505064
Epoch 46/50, Sequence 451/2944, Loss: 0.0013329412322491407
Epoch 46/50, Sequence 476/2944, Loss: 0.0009347519371658564
Epoch 46/50, Sequence 501/2944, Loss: 0.0006598661420866847
Epoch 46/50, Sequence 526/2944, Loss: 0.00046103796921670437
Epoch 46/50, Sequence 551/2944, Loss: 0.0003225886030122638
Epoch 46/50, Sequence 576/2944, Loss: 0.00022709774202667177
Epoch 46/50, Sequence 601/2944, Loss: 0.0001576741924509406
Epoch 46/50, Sequence 626/2944, Loss: 0.00010700232087401673
Epoch 46/50, Sequence 651/2944, Loss: 7.07109211361967e-05
Epoch 46/50, Sequence 676/2944, Loss: 4.872143836109899e-05
Epoch 46/50, Sequence 701/2944, Loss: 3.355

Epoch 47/50, Sequence 751/2944, Loss: 1.4956778613850474e-05
Epoch 47/50, Sequence 776/2944, Loss: 9.326087820227258e-06
Epoch 47/50, Sequence 801/2944, Loss: 5.466687980515417e-06
Epoch 47/50, Sequence 826/2944, Loss: 2.9025204639765434e-06
Epoch 47/50, Sequence 851/2944, Loss: 1.4640098697782378e-06
Epoch 47/50, Sequence 876/2944, Loss: 1.0334106264053844e-06
Epoch 47/50, Sequence 901/2944, Loss: 7.797460739311646e-07
Epoch 47/50, Sequence 926/2944, Loss: 5.734825663239462e-07
Epoch 47/50, Sequence 951/2944, Loss: 3.9477285440625565e-07
Epoch 47/50, Sequence 976/2944, Loss: 2.5693722705000255e-07
Epoch 47/50, Sequence 1001/2944, Loss: 1.5987313872756204e-07
Epoch 47/50, Sequence 1026/2944, Loss: 1.0032152175654119e-07
Epoch 47/50, Sequence 1051/2944, Loss: 6.040419009423204e-08
Epoch 47/50, Sequence 1076/2944, Loss: 4.627863958717171e-08
Epoch 47/50, Sequence 1101/2944, Loss: 8.310381360843166e-08
Epoch 47/50, Sequence 1126/2944, Loss: 1.5072133408011723e-07
Epoch 47/50, Sequence 115

Epoch 48/50, Sequence 1201/2944, Loss: 4.76199460308635e-07
Epoch 48/50, Sequence 1226/2944, Loss: 3.1714498049950635e-07
Epoch 48/50, Sequence 1251/2944, Loss: 1.7506094707187003e-07
Epoch 48/50, Sequence 1276/2944, Loss: 1.0224107427347917e-07
Epoch 48/50, Sequence 1301/2944, Loss: 7.930226075814062e-08
Epoch 48/50, Sequence 1326/2944, Loss: 3.955998906235436e-08
Epoch 48/50, Sequence 1351/2944, Loss: 3.486833222154928e-08
Epoch 48/50, Sequence 1376/2944, Loss: 1.2165990881385369e-08
Epoch 48/50, Sequence 1401/2944, Loss: 1.4755794808252176e-08
Epoch 48/50, Sequence 1426/2944, Loss: 3.074031695859958e-09
Epoch 48/50, Sequence 1451/2944, Loss: 1.2427081585997257e-08
Epoch 48/50, Sequence 1476/2944, Loss: 2.1084491930167815e-09
Epoch 48/50, Sequence 1501/2944, Loss: 1.5994199298319245e-09
Epoch 48/50, Sequence 1526/2944, Loss: 1.6130398128311185e-09
Epoch 48/50, Sequence 1551/2944, Loss: 1.049797360153093e-09
Epoch 48/50, Sequence 1576/2944, Loss: 3.7075764680594148e-09
Epoch 48/50, Se

Epoch 49/50, Sequence 1651/2944, Loss: 1.1391776411073806e-10
Epoch 49/50, Sequence 1676/2944, Loss: 3.7988220902285263e-10
Epoch 49/50, Sequence 1701/2944, Loss: 1.1342382588708233e-09
Epoch 49/50, Sequence 1726/2944, Loss: 2.5431947658915988e-09
Epoch 49/50, Sequence 1751/2944, Loss: 3.0032474285235367e-09
Epoch 49/50, Sequence 1776/2944, Loss: 2.127924059180941e-09
Epoch 49/50, Sequence 1801/2944, Loss: 3.288888050789751e-09
Epoch 49/50, Sequence 1826/2944, Loss: 2.658249398379553e-09
Epoch 49/50, Sequence 1851/2944, Loss: 2.362850803905303e-09
Epoch 49/50, Sequence 1876/2944, Loss: 5.56204582391473e-10
Epoch 49/50, Sequence 1901/2944, Loss: 2.479313421233087e-09
Epoch 49/50, Sequence 1926/2944, Loss: 8.963378650417297e-11
Epoch 49/50, Sequence 1951/2944, Loss: 2.951440647436243e-09
Epoch 49/50, Sequence 1976/2944, Loss: 2.883596916802844e-09
Epoch 49/50, Sequence 2001/2944, Loss: 1.1985554215776517e-10
Epoch 49/50, Sequence 2026/2944, Loss: 0.0001636050146771595
Epoch 49/50, Sequen

Epoch 50/50, Sequence 2101/2944, Loss: 0.10659423470497131
Epoch 50/50, Sequence 2126/2944, Loss: 0.18050289154052734
Epoch 50/50, Sequence 2151/2944, Loss: 0.11769354343414307
Epoch 50/50, Sequence 2176/2944, Loss: 0.08453654497861862
Epoch 50/50, Sequence 2201/2944, Loss: 0.05418448895215988
Epoch 50/50, Sequence 2226/2944, Loss: 0.03304530680179596
Epoch 50/50, Sequence 2251/2944, Loss: 0.019791848957538605
Epoch 50/50, Sequence 2276/2944, Loss: 0.011767248623073101
Epoch 50/50, Sequence 2301/2944, Loss: 0.007226698100566864
Epoch 50/50, Sequence 2326/2944, Loss: 0.0046190181747078896
Epoch 50/50, Sequence 2351/2944, Loss: 0.003093354171141982
Epoch 50/50, Sequence 2376/2944, Loss: 0.0021923871245235205
Epoch 50/50, Sequence 2401/2944, Loss: 0.0016244191210716963
Epoch 50/50, Sequence 2426/2944, Loss: 0.0012373115168884397
Epoch 50/50, Sequence 2451/2944, Loss: 0.0009613210568204522
Epoch 50/50, Sequence 2476/2944, Loss: 0.000757591042201966
Epoch 50/50, Sequence 2501/2944, Loss: 0.

In [46]:
predicted_array_1 = np.squeeze(np.array(predicted_vectors))

df_pred_1 = pd.DataFrame(predicted_array_1, columns=['velocity_data_x', 'velocity_data_y', 'velocity_data_z'])

In [47]:
df_pred_1

,velocity_data_x,velocity_data_y,velocity_data_z
0,0.165763,0.429384,0.586409
1,0.166215,0.429221,0.585888
2,0.167070,0.428940,0.584993
3,0.168265,0.428581,0.583850
4,0.169746,0.428179,0.582566
...,...,...,...
2939,0.165879,0.429386,0.586409
2940,0.165855,0.429386,0.586409
2941,0.165832,0.429385,0.586409
2942,0.165809,0.429385,0.586409


In [48]:
model_path = 'saved_model_LSTM.pth'

torch.save({
    'epoch': epoch,
    'model_state_dict': model_1.state_dict(),
    'optimizer_state_dict': optimizer_1.state_dict(),
    'loss': loss_forward_1,
}, model_path)

print("Model saved successfully!")

Model saved successfully!


In [49]:
num_sequences = last_index // 4 
y = torch.zeros(num_sequences)
y_hat = torch.zeros(num_sequences)

In [50]:
std_dev = data.std()
std_dev_array = np.array(std_dev).reshape(1, -1)
std_dev_array

array([[0.35271156, 0.28579752, 0.00593083]])

In [51]:
deviation_threshold = np.array(3 * std_dev).reshape(1, -1)
deviation_threshold
deviation_threshold_1=deviation_threshold[0][0]
deviation_threshold_2=deviation_threshold[0][1]
deviation_threshold_3=deviation_threshold[0][2]

In [52]:
for i in range(len(df_pred_1) - 4):
    if (abs(df_pred_1['velocity_data_x'][i] - data['velocity_data_y'][i + 4]) > deviation_threshold_1 or
        abs(df_pred_1['velocity_data_y'][i] - data['velocity_data_x'][i + 4]) > deviation_threshold_2 or
        abs(df_pred_1['velocity_data_z'][i] - data['velocity_data_z'][i + 4]) > deviation_threshold_3):
        y_hat[i]=1


In [53]:
y_hat

tensor([1., 1., 1.,  ..., 0., 0., 0.])

In [54]:
test_data=pd.read_csv('Dataset/TC2/._test.csv')

In [55]:
test_data

,velocity_data_x,velocity_data_y,velocity_data_z
0,-1.212264,0.883470,-0.000159
1,-1.212274,0.883475,-0.000159
2,-1.212285,0.883480,-0.000159
3,-1.212296,0.883486,-0.000159
4,-1.212306,0.883491,-0.000159
...,...,...,...
1995,-1.499317,8.726109,-0.000160
1996,-1.499319,0.082326,-0.000160
1997,-1.499320,37.618766,-0.000160
1998,-1.499322,1.983398,-0.000160


In [56]:
sequence_size = 4
sequences_test = []

for i in range(len(test_data) - sequence_size + 1):
    sequence_test = test_data.iloc[i:i + sequence_size]
    sequences_test.append(sequence_test)

sequences_test_df = pd.concat(sequences_test, ignore_index=True)

sequences_test_df

,velocity_data_x,velocity_data_y,velocity_data_z
0,-1.212264,0.883470,-0.000159
1,-1.212274,0.883475,-0.000159
2,-1.212285,0.883480,-0.000159
3,-1.212296,0.883486,-0.000159
4,-1.212274,0.883475,-0.000159
...,...,...,...
7983,-1.499322,1.983398,-0.000160
7984,-1.499319,0.082326,-0.000160
7985,-1.499320,37.618766,-0.000160
7986,-1.499322,1.983398,-0.000160


In [57]:
model_path = 'saved_model_LSTM.pth'
checkpoint = torch.load(model_path)

In [58]:
model = LSTMModel_forward(input_size, hidden_size, num_layers, dropout)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

C:\Users\Archi\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


LSTMModel_forward(
  (lstm_forward): LSTM(3, 64, batch_first=True, dropout=0.1)
  (fc1_forward): Linear(in_features=64, out_features=64, bias=True)
  (relu_forward): ReLU()
  (fc2_forward): Linear(in_features=64, out_features=3, bias=True)
)

In [59]:
x_test = sequences_test_df.values.tolist()
x_test = torch.tensor(x_test).to(device)

In [60]:
with torch.no_grad():
    predicted_vectors_test = []
    for idx in range(0, len(x_test), sequence_size):
        sequence_test = x_test[idx:idx + sequence_size].to(device)  

        out_forward_test = model(sequence_test)
        predicted_next_element_test = out_forward_test[-1, :].view(-1, 3)
        predicted_vectors_test.append(predicted_next_element_test.cpu().numpy())


In [61]:
predicted_vectors_test

[array([[0.15666665, 0.5000601 , 0.5659871 ]], dtype=float32),
 array([[0.15666649, 0.50006145, 0.56598675]], dtype=float32),
 array([[0.15666635, 0.50006264, 0.5659864 ]], dtype=float32),
 array([[0.15666622, 0.50006384, 0.56598604]], dtype=float32),
 array([[0.1566661 , 0.50006497, 0.5659857 ]], dtype=float32),
 array([[0.15666604, 0.5000661 , 0.5659853 ]], dtype=float32),
 array([[0.15666604, 0.5000671 , 0.565985  ]], dtype=float32),
 array([[0.15666614, 0.50006795, 0.5659848 ]], dtype=float32),
 array([[0.15666634, 0.50006855, 0.5659846 ]], dtype=float32),
 array([[0.15666664, 0.5000688 , 0.56598455]], dtype=float32),
 array([[0.15666707, 0.50006884, 0.56598455]], dtype=float32),
 array([[0.15666763, 0.50006855, 0.5659847 ]], dtype=float32),
 array([[0.15666829, 0.50006783, 0.56598496]], dtype=float32),
 array([[0.15666902, 0.50006694, 0.56598526]], dtype=float32),
 array([[0.15666984, 0.5000657 , 0.5659857 ]], dtype=float32),
 array([[0.15667072, 0.50006425, 0.56598616]], dtype=fl

In [62]:
predicted_test_array = np.squeeze(np.array(predicted_vectors_test))

df_test_pred = pd.DataFrame(predicted_test_array, columns=['velocity_data_x', 'velocity_data_y', 'velocity_data_z'])

In [63]:
df_test_pred

,velocity_data_x,velocity_data_y,velocity_data_z
0,0.156667,0.500060,0.565987
1,0.156666,0.500061,0.565987
2,0.156666,0.500063,0.565986
3,0.156666,0.500064,0.565986
4,0.156666,0.500065,0.565986
...,...,...,...
1992,0.385952,0.361291,0.561611
1993,0.152034,0.511719,0.564961
1994,0.491273,0.331482,0.540126
1995,0.167562,0.434010,0.587144


In [64]:
num_sequences_1 = len(sequences_test_df) // 4 
y_test = torch.zeros(num_sequences_1)
y_hat_test = torch.zeros(num_sequences_1)

In [65]:
std_dev_1 = test_data.std()
std_dev_array_1 = np.array(std_dev_1).reshape(1, -1)
std_dev_array_1

array([[2.00598262e-01, 1.64150534e+01, 4.48844794e-03]])

In [66]:
deviation_threshold_test = np.array(3 * std_dev).reshape(1, -1)
deviation_threshold_test
deviation_threshold_test_1=deviation_threshold_test[0][0]
deviation_threshold_test_2=deviation_threshold_test[0][1]
deviation_threshold_test_3=deviation_threshold_test[0][2]

In [67]:
for i in range(len(df_test_pred)):
    if (abs(df_test_pred['velocity_data_x'][i] - test_data['velocity_data_x'][i + 3]) > deviation_threshold_test_1 or
        abs(df_test_pred['velocity_data_y'][i] - test_data['velocity_data_y'][i + 3]) > deviation_threshold_test_2 or
        abs(df_test_pred['velocity_data_z'][i] - test_data['velocity_data_z'][i + 3]) > deviation_threshold_test_3):
        y_hat_test[i]=1


In [68]:
y_hat_test

tensor([1., 1., 1.,  ..., 1., 1., 1.])

In [69]:
test_data_1=pd.read_csv('Dataset/TC2/test.csv')
column_4_values = test_data_1.iloc[3:, 3].values

In [70]:
column_4_values
column_4_values.shape

(1997,)

In [71]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(column_4_values, y_hat_test)
print("Accuracy:", accuracy)


Accuracy: 0.7511266900350526
